In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../zkstats/core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.round(np.random.uniform(min(data), max(data), len(data)),2)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.tensor(statistics.mode(dummy_data))



In [5]:
theory_output = torch.tensor(statistics.mode(data))
print("theory mode: ", theory_output)

theory mode:  tensor(4.1000)


In [6]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
    def forward(self,X):
        # some expression of tolerance to error in the inference
        count_equal = torch.sum((torch.abs(X-self.w)<0.01*self.w).double())
        result = torch.zeros(X.size()[1])
        index_array = torch.tensor(range(X.size()[1]))

        for index in index_array:
            ele = X[0][index][0]
            if (torch.sum((torch.abs(X-ele)<0.01*ele).double())<=count_equal):
                check_bool = torch.tensor(True)
            else:
                check_bool = torch.tensor(False)
            result[index] = check_bool

        return (torch.sum(result) == X.size()[1], self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

dummy output:  tensor(8.7200, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3192496014.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3192496014.py:14: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3192496014.py:16: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

In [7]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(statistics.mode(data))

print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        count_equal = torch.sum((torch.abs(X-self.w)<0.01*self.w).double())
        result = torch.zeros(X.size()[1])
        index_array = torch.tensor(range(X.size()[1]))
        
        for index in index_array:
            ele = X[0][index][0]
            if (torch.sum((torch.abs(X-ele)<0.01*ele).double())<=count_equal):
                check_bool = torch.tensor(True)
            else:
                check_bool = torch.tensor(False)
            result[index] = check_bool

        return (torch.sum(result) == X.size()[1], self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory_output:  tensor(4.1000)
==== Generate & Calibrate Setting ====


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3050767087.py:16: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(X.size()[1]))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3050767087.py:18: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for index in index_array:
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_41683/3050767087.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so th

scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":7,"param_scale":7,"scale_rebase_multiplier":10,"lookup_range":[0,0],"logrows":11,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":1312,"total_assignments":8,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,7],"model_input_scales":[7],"module_sizes":{"kzg":[],"poseidon":[1312,[1]],"elgamal":[0,[0]]},"required_lookups":[],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 0.24821925163269043 seconds


spawning module 0
spawning module 2


Theory output:  tensor(4.1000)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 4.1015625
==== Generating Proof ====
proof:  {'instances': [[[8968413016118640008, 13978084904548187999, 5732603028781509343, 2742516011951746503], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [11664057200304715047, 16495917767961733174, 11618098729275913590, 1146451785665059695]]], 'proof': '2a7928514c8ec641c5c842933cbba36f104ebf95a18453a42131fd6f327e78881b76d6964845c2f05978e8dd5db95912b76398289cdce803d578dbd5f12a5a980665751da460434bedc03e65a65be1606eb0f1e953cd3a74d2aa8e81211d6a9622b59b28504b04442047f3c66760cf944319a996c329a3faf4a569d02f7721a929d5220f8aca863fdbc768add8554bab13f143d0888be4086210e0fcc04679ca04677702c3137565f413d6bde53dc4542444deceee772797b4baa35b38e0ab3101e9b91f6c7c32744614edbdf908df61e8b988d7d908ce85d46f12a930c40f8e1c63a1f992138d01e50940d8efe31d7c41e1ad73b0a7bd5b5324415852fce25706c4a4ae4c0c0a4479653c22c9e456b9dd229e759fea9a24c64

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[8968413016118640008, 13978084904548187999, 5732603028781509343, 2742516011951746503], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [11664057200304715047, 16495917767961733174, 11618098729275913590, 1146451785665059695]]]
proof boolean:  1.0
proof result 1 : 4.1015625
verified
